In [1]:
import time
from pathlib import Path

import matplotlib.pyplot as plt
from matplotlib import gridspec
import matplotlib.patches as mpatches
import numpy as np
import tensorflow as tf


import pandas as pd
import seaborn as sns
from sklearn import preprocessing

In [2]:
try:
    # enable all GPUS
    tf.config.set_visible_devices([], 'GPU')
    visible_devices = tf.config.get_visible_devices()
    for device in visible_devices:
        assert device.device_type == 'GPU'
except:
    # Invalid device or cannot modify virtual devices once initialized.
    pass

In [3]:
DATA_DIR_SLOW = "/Users/kedi/Desktop/polito lectures and notes/ML in App/project/KukaVelocityDataset/KukaSlow.npy"
DATA_DIR_NORMAL = "/Users/kedi/Desktop/polito lectures and notes/ML in App/project/KukaVelocityDataset/KukaNormal.npy"
DATA_DIR_C_NAMES = "/Users/kedi/Desktop/polito lectures and notes/ML in App/project/KukaVelocityDataset/KukaColumnNames.npy"

# Set style for matplotlib
plt.style.use("Solarize_Light2")

In [4]:
data_normal = np.load(DATA_DIR_NORMAL)
data_slow = np.load(DATA_DIR_SLOW)
column_names = np.load(DATA_DIR_C_NAMES)

In [119]:
print(column_names)

['action' 'machine_nameKuka Robot_apparent_power'
 'machine_nameKuka Robot_current' 'machine_nameKuka Robot_frequency'
 'machine_nameKuka Robot_phase_angle' 'machine_nameKuka Robot_power'
 'machine_nameKuka Robot_power_factor'
 'machine_nameKuka Robot_reactive_power' 'machine_nameKuka Robot_voltage'
 'sensor_id1_AccX' 'sensor_id1_AccY' 'sensor_id1_AccZ' 'sensor_id1_GyroX'
 'sensor_id1_GyroY' 'sensor_id1_GyroZ' 'sensor_id1_q1' 'sensor_id1_q2'
 'sensor_id1_q3' 'sensor_id1_q4' 'sensor_id1_temp' 'sensor_id2_AccX'
 'sensor_id2_AccY' 'sensor_id2_AccZ' 'sensor_id2_GyroX' 'sensor_id2_GyroY'
 'sensor_id2_GyroZ' 'sensor_id2_q1' 'sensor_id2_q2' 'sensor_id2_q3'
 'sensor_id2_q4' 'sensor_id2_temp' 'sensor_id3_AccX' 'sensor_id3_AccY'
 'sensor_id3_AccZ' 'sensor_id3_GyroX' 'sensor_id3_GyroY'
 'sensor_id3_GyroZ' 'sensor_id3_q1' 'sensor_id3_q2' 'sensor_id3_q3'
 'sensor_id3_q4' 'sensor_id3_temp' 'sensor_id4_AccX' 'sensor_id4_AccY'
 'sensor_id4_AccZ' 'sensor_id4_GyroX' 'sensor_id4_GyroY'
 'sensor_id4_GyroZ

In [120]:
# Normalization train on train set transform both
scaler = preprocessing.StandardScaler().fit(data_normal)
data_train_scaled = scaler.transform(data_normal)
data_test_scaled = scaler.transform(data_slow[:,:-1]) #

In [121]:
#windows of 50 with stride
width_window = 50
data_train = np.array([data_train_scaled[i:i+width_window,:] for i in range(0, len(data_train_scaled)-width_window)])
data_test = np.array([data_test_scaled[i:i+width_window,:] for i in range(0, len(data_test_scaled)-width_window)])
print(f"train data shape: {data_train.shape} \n test data shape: {data_test.shape} ")

train data shape: (233742, 50, 86) 
 test data shape: (41488, 50, 86) 


In [122]:
# take part of data to gather results faster
percentage = 0.10
data_train = data_train[0:int(percentage*data_train_scaled.shape[0]),:]
data_test = data_test[0:int(percentage*data_test_scaled.shape[0]),:]
print(f"train data shape: {data_train.shape} \n test data shape: {data_test.shape} ")

train data shape: (23379, 50, 86) 
 test data shape: (4153, 50, 86) 


In [123]:
batch_size = 256
train_buf = 60000
comment = False

train_dataset = tf.data.Dataset.from_tensor_slices(data_train)
train_dataset = train_dataset.shuffle(buffer_size=train_buf)
train_dataset = train_dataset.batch(batch_size)

In [124]:
def make_encoder_model(z_size):
    inputs = tf.keras.layers.Input(shape=(50, 86))
    if comment is True : print(f"encoder input: {inputs.shape}")
    x = tf.keras.layers.Conv1D(filters=128, kernel_size=7, strides=2, padding='same')(inputs)
    x = tf.keras.layers.LeakyReLU(0.2)(x)
    x = tf.keras.layers.Conv1D(filters=128, kernel_size=7, strides=2, padding='same')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.LeakyReLU(0.2)(x)
    x = tf.keras.layers.Conv1D(filters=256, kernel_size=7, strides=2, padding='same')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.LeakyReLU(0.2)(x)
    x = tf.keras.layers.Conv1D(filters=256, kernel_size=7, strides=2, padding='same')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.LeakyReLU(0.2)(x)
    x = tf.keras.layers.Conv1D(filters=256, kernel_size=7, strides=2, padding='same')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.LeakyReLU(0.2)(x)
    z = tf.keras.layers.Conv1D(filters=z_size, kernel_size=7, strides=2, padding='same')(x)
    if comment is True : print(f"endoder output: {z.shape}")

    model = tf.keras.Model(inputs=inputs, outputs=z)
    return model

def make_decoder_model(z_size):
    encoded = tf.keras.Input(shape=(1, z_size))
    if comment is True : print(f"decoder input: {encoded.shape}")

    x = tf.keras.layers.Conv1D(256, kernel_size=7, padding='same', activation='relu')(encoded)
    x = tf.keras.layers.UpSampling1D(2)(x)
    x = tf.keras.layers.Conv1D(256, kernel_size=7, padding='same', activation='relu')(x)
    x = tf.keras.layers.UpSampling1D(2)(x)
    x = tf.keras.layers.Conv1D(256, kernel_size=7, padding='same', activation='relu')(x)
    x = tf.keras.layers.UpSampling1D(2)(x)
    x = tf.keras.layers.Conv1D(128, kernel_size=7, padding='same', activation='relu')(x)
    x = tf.keras.layers.UpSampling1D(2)(x)
    x = tf.keras.layers.Conv1D(128, kernel_size=7, padding='same', activation='relu')(x)
    x = tf.keras.layers.UpSampling1D(2)(x)
    x = tf.keras.layers.Conv1D(86, kernel_size=7, activation='relu')(x)
    x = tf.keras.layers.UpSampling1D(2)(x)

    reconstruction = tf.keras.layers.Conv1D(filters=86, kernel_size=3, activation='sigmoid')(x)
    if comment is True : print(f"decoder output: {reconstruction.shape}")
    decoder = tf.keras.Model(inputs=encoded, outputs=reconstruction)
    return decoder

def make_discriminator_model(z_size):
    encoded = tf.keras.Input(shape=(1, z_size))
    if comment is True : print(f"discriminator input: {encoded.shape}")
    x = tf.keras.layers.Dense(128)(encoded)
    x = tf.keras.layers.LeakyReLU(0.2)(x)
    x = tf.keras.layers.Dense(128)(x)
    x = tf.keras.layers.LeakyReLU(0.2)(x)
    prediction = tf.keras.layers.Dense(1)(x)
    if comment is True : print(f"discriminator output: {prediction.shape}")
    model = tf.keras.Model(inputs=encoded, outputs=prediction)
    return model

In [125]:
z_dim = 512
with tf.device('/GPU:0'):
    encoder = make_encoder_model(z_dim)
    decoder = make_decoder_model(z_dim)
    discriminator = make_discriminator_model(z_dim)

In [126]:
# Define loss functions
ae_loss_weight = 1.
gen_loss_weight = 1.
dc_loss_weight = 1.

with tf.device('/GPU:0'):
    cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)
    mse = tf.keras.losses.MeanSquaredError()
    accuracy = tf.keras.metrics.BinaryAccuracy()

In [127]:
def autoencoder_loss(inputs, reconstruction, loss_weight):
    return loss_weight * mse(inputs, reconstruction)


def discriminator_loss(real_output, fake_output, loss_weight):
    loss_real = cross_entropy(tf.ones_like(real_output), real_output)
    loss_fake = cross_entropy(tf.zeros_like(fake_output), fake_output)
    return loss_weight * (loss_fake + loss_real)


def generator_loss(fake_output, loss_weight):
    return loss_weight * cross_entropy(tf.ones_like(fake_output), fake_output)

In [128]:
base_lr = 0.00025
max_lr = 0.0025

n_samples = data_train.shape[0]
step_size = 2 * np.ceil(n_samples / batch_size)
global_step = 0

In [129]:
with tf.device('/GPU:0'):
    ae_optimizer = tf.keras.optimizers.Adam(learning_rate=base_lr)
    dc_optimizer = tf.keras.optimizers.Adam(learning_rate=base_lr)
    gen_optimizer = tf.keras.optimizers.Adam(learning_rate=base_lr)

In [130]:
def train_step(batch_x):
    with tf.device('/GPU:0'):
        # -------------------------------------------------------------------------------------------------------------
        # Autoencoder
        with tf.GradientTape() as ae_tape:
            encoder_output = encoder(batch_x, training=True)
            decoder_output = decoder(encoder_output, training=True)

            if comment is True : print(f"Train step: \nbatch_x: {batch_x.shape}, decoder_output: {decoder_output.shape}")
            # Autoencoder loss
            ae_loss = autoencoder_loss(batch_x, decoder_output, ae_loss_weight)

        ae_grads = ae_tape.gradient(ae_loss, encoder.trainable_variables + decoder.trainable_variables)
        ae_optimizer.apply_gradients(zip(ae_grads, encoder.trainable_variables + decoder.trainable_variables))

        # -------------------------------------------------------------------------------------------------------------
        # Discriminator
        with tf.GradientTape() as dc_tape:
            real_distribution = tf.random.normal([batch_x.shape[0], 1, z_dim], mean=0.0, stddev=1.0)
            encoder_output = encoder(batch_x, training=True)

            dc_real = discriminator(real_distribution, training=True)
            dc_fake = discriminator(encoder_output, training=True)

            # Discriminator Loss
            dc_loss = discriminator_loss(dc_real, dc_fake, dc_loss_weight)

            # Discriminator Acc
            dc_acc = accuracy(tf.concat([tf.ones_like(dc_real), tf.zeros_like(dc_fake)], axis=0),
                              tf.concat([dc_real, dc_fake], axis=0))

        dc_grads = dc_tape.gradient(dc_loss, discriminator.trainable_variables)
        dc_optimizer.apply_gradients(zip(dc_grads, discriminator.trainable_variables))

        # -------------------------------------------------------------------------------------------------------------
        # Generator (Encoder)
        with tf.GradientTape() as gen_tape:
            encoder_output = encoder(batch_x, training=True)
            dc_fake = discriminator(encoder_output, training=True)

            # Generator loss
            gen_loss = generator_loss(dc_fake, gen_loss_weight)

        gen_grads = gen_tape.gradient(gen_loss, encoder.trainable_variables)
        gen_optimizer.apply_gradients(zip(gen_grads, encoder.trainable_variables))

        return ae_loss, dc_loss, dc_acc, gen_loss

In [131]:
with tf.device('/GPU:0'):
    n_epochs = 100
    for epoch in range(n_epochs):
        start = time.time()

        # Learning rate schedule
        if epoch in [60, 100, 300]:
            base_lr = base_lr / 2
            max_lr = max_lr / 2
            step_size = step_size / 2

            print('learning rate changed!')

        epoch_ae_loss_avg = tf.metrics.Mean()
        epoch_dc_loss_avg = tf.metrics.Mean()
        epoch_dc_acc_avg = tf.metrics.Mean()
        epoch_gen_loss_avg = tf.metrics.Mean()
        for batch, (batch_x) in enumerate(train_dataset):
            # -------------------------------------------------------------------------------------------------------------
            # Calculate cyclic learning rate
            global_step = global_step + 1
            cycle = np.floor(1 + global_step / (2 * step_size))
            x_lr = np.abs(global_step / step_size - 2 * cycle + 1)
            clr = base_lr + (max_lr - base_lr) * max(0, 1 - x_lr)
            ae_optimizer.lr = clr
            dc_optimizer.lr = clr
            gen_optimizer.lr = clr

            ae_loss, dc_loss, dc_acc, gen_loss = train_step(batch_x)

            epoch_ae_loss_avg(ae_loss)
            epoch_dc_loss_avg(dc_loss)
            epoch_dc_acc_avg(dc_acc)
            epoch_gen_loss_avg(gen_loss)

        epoch_time = time.time() - start
        print('{:4d}: TIME: {:.2f} ETA: {:.2f} AE_LOSS: {:.4f} DC_LOSS: {:.4f} DC_ACC: {:.4f} GEN_LOSS: {:.4f}' \
              .format(epoch, epoch_time,
                      epoch_time * (n_epochs - epoch),
                      epoch_ae_loss_avg.result(),
                      epoch_dc_loss_avg.result(),
                      epoch_dc_acc_avg.result(),
                      epoch_gen_loss_avg.result()))

        # -------------------------------------------------------------------------------------------------------------
        """
        if epoch % 10 == 0:
            # Latent Space
            x_test_encoded = encoder(x_test, training=False)
            label_list = list(y_test)

            fig = plt.figure()
            classes = set(label_list)
            colormap = plt.cm.rainbow(np.linspace(0, 1, len(classes)))
            kwargs = {'alpha': 0.8, 'c': [colormap[i] for i in label_list]}
            ax = plt.subplot(111, aspect='equal')
            box = ax.get_position()
            ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])
            handles = [mpatches.Circle((0, 0), label=class_, color=colormap[i])
                       for i, class_ in enumerate(classes)]
            ax.legend(handles=handles, shadow=True, bbox_to_anchor=(1.05, 0.45),
                      fancybox=True, loc='center left')
            plt.scatter(x_test_encoded[:, :, :, 0], x_test_encoded[:, :, :, 1], s=2, **kwargs)
            ax.set_xlim([-3, 3])
            ax.set_ylim([-3, 3])

            plt.savefig(latent_space_dir / ('epoch_%d.png' % epoch))
            plt.close('all')

            # Reconstruction
            n_digits = 20  # how many digits we will display
            x_test_decoded = decoder(encoder(x_test[:n_digits], training=False), training=False)
            x_test_decoded = np.reshape(x_test_decoded, [-1, 28, 28]) * 255
            fig = plt.figure(figsize=(20, 4))
            for i in range(n_digits):
                # display original
                ax = plt.subplot(2, n_digits, i + 1)
                plt.imshow(x_test[i].reshape(28, 28))
                plt.gray()
                ax.get_xaxis().set_visible(False)
                ax.get_yaxis().set_visible(False)

                # display reconstruction
                ax = plt.subplot(2, n_digits, i + 1 + n_digits)
                plt.imshow(x_test_decoded[i])
                plt.gray()
                ax.get_xaxis().set_visible(False)
                ax.get_yaxis().set_visible(False)

            plt.savefig(reconstruction_dir / ('epoch_%d.png' % epoch))
            plt.close('all')

            # Sampling
            x_points = np.linspace(-3, 3, 20).astype(np.float32)
            y_points = np.linspace(-3, 3, 20).astype(np.float32)

            nx, ny = len(x_points), len(y_points)
            plt.subplot()
            gs = gridspec.GridSpec(nx, ny, hspace=0.05, wspace=0.05)

            for i, g in enumerate(gs):
                z = np.concatenate(([x_points[int(i / ny)]], [y_points[int(i % nx)]]))
                z = np.reshape(z, (1, 1, 1, 2))
                x = decoder(z, training=False).numpy()
                ax = plt.subplot(g)
                img = np.array(x.tolist()).reshape(28, 28)
                ax.imshow(img, cmap='gray')
                ax.set_xticks([])
                ax.set_yticks([])
                ax.set_aspect('auto')
            plt.savefig(sampling_dir / ('epoch_%d.png' % epoch))
            plt.close('all')
        """

   0: TIME: 61.78 ETA: 6178.24 AE_LOSS: 1.0766 DC_LOSS: 3.5214 DC_ACC: 0.2972 GEN_LOSS: 2.3127
   1: TIME: 59.95 ETA: 5935.42 AE_LOSS: 1.0393 DC_LOSS: 1.8571 DC_ACC: 0.4002 GEN_LOSS: 1.9941
   2: TIME: 59.50 ETA: 5831.23 AE_LOSS: 1.0379 DC_LOSS: 1.1423 DC_ACC: 0.4812 GEN_LOSS: 3.4687
   3: TIME: 60.00 ETA: 5819.82 AE_LOSS: 1.0380 DC_LOSS: 1.0224 DC_ACC: 0.5592 GEN_LOSS: 1.6385
   4: TIME: 60.17 ETA: 5776.49 AE_LOSS: 1.0381 DC_LOSS: 0.9180 DC_ACC: 0.6092 GEN_LOSS: 1.8378
   5: TIME: 59.56 ETA: 5658.57 AE_LOSS: 1.0380 DC_LOSS: 1.6868 DC_ACC: 0.6330 GEN_LOSS: 2.6973


KeyboardInterrupt: 